# Applied Data Science Capstone Project

## Examining Conditions To Open A Specific Restaurant In Hamburg, Germany!

### 1 Outlining

### 1.1 Introduction


After Berlin Hamburg is the largest city in Germany and largest non-capital city in the European Union with almost <b>1.9 million inhabitants</b>. It is known for it is long rich history dating back more than 1,000 years, famous architecture (e.g. Elbphilharmonie, UNESCO awarded Speicherstadt etc.) and it is famous night life, like the Reeperbahn. Furthermore Hamburg is very diverse. On the one hand there are in comparison poor boroughs with small income and high unemployment rate while some boroughs are extraordinary wealthy. The true potential lies in the density of the city. With <b>2,453 people per square meter</b> and more than <b>300,000 commuters</b> every day it has potential customers that their belly want to be filled on a regular basis. The exciting part for a data scientist is to analyze with Foursquare the ideal location within Hamburg and what kind of cuisine should be offered when the restaurant is opening.

### 1.2 Business Problem/ Approach

Next to the cuisine skills all economic options should be considered when opening a restaurant. The lack of information regarding the location, competitors in the neighborhood and appeal of them can be reduced by using the Foursquare API. An similiar known approach like New York City and Toronto will be chosen for the destination Hamburg, Germany. After extracting the data for Hamburg from Foursquare an examination will be done as follows:

    1. What is a suitable location?
    2. What kind of cuisine should be offered?
    3. What are necessary informations about the chosen location?
    
In the end a summary will give a recommendation of a location and cuisine style (e.g. Afghan, Asian, Italian etc.).

### 2 Data Source

First of all a dataset of Hamburg will be built which provides information on boroughs, neighborhoods and corresponding data. After that the Foursquare API will be used to analyze the situation in each borough and neighborhood. A conclusion where the findings of the created data will be provided. 

### 2.1 Setting up the environment

Importing necessary libraries

In [ ]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library
# additionally importing beautiful soup library

from bs4 import BeautifulSoup